In [ ]:
!pip install --upgrade boto3 sagemaker

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session() #this would automaticall create a sagemeaker bucket if you dont have one and its were upload of logs, data and models would be stored

sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
#this if statement is actually redundant and only needed if the sagemeker session buvket was set to something other than none

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

# Hub model configuration <https://huggingface.co/models>
hub = {
  #'HF_MODEL_ID':'openai/gpt-oss-20b', # model_id from hf.co/models
  #'HF_TASK':'question-answering'                           # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                                                
   role=role,                                              
   transformers_version="4.26",                            
   pytorch_version="1.13",                                 
   py_version='py39',                                      
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge" #VERY EXPENSIVE SO MAKE SURE TO DESTROY WHEN NOT IN USE
)